In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)
    
    
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)
        
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")
            
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occured while scraping bus details for {url}: {str(e)}")
            return[]
        
    except Exception as e:
        print(f"Error occured while accessing {url}: {str(e)}")
        return []
    
all_bus_details = []

def scrape_all_pages():
    for page in range(1, 6):
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)
                
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
                    
                    
                    
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")
            
scrape_all_pages()

df = pd.DataFrame(all_bus_details)

df.to_csv('uttra_pradesh_bus_details.csv', index = False)

driver.quit()

Error occured while scraping bus details for https://www.redbus.in/bus-tickets/lucknow-to-aligarh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF63766FDA5+29557]
	(No symbol) [0x00007FF6375E2240]
	(No symbol) [0x00007FF63749B6EA]
	(No symbol) [0x00007FF6374EFA15]
	(No symbol) [0x00007FF6374EFC6C]
	(No symbol) [0x00007FF63753BB07]
	(No symbol) [0x00007FF63751753F]
	(No symbol) [0x00007FF6375388A3]
	(No symbol) [0x00007FF6375172A3]
	(No symbol) [0x00007FF6374E12DF]
	(No symbol) [0x00007FF6374E2451]
	GetHandleVerifier [0x00007FF63799DCBD+3363469]
	GetHandleVerifier [0x00007FF6379E9B47+3674391]
	GetHandleVerifier [0x00007FF6379DEAEB+3629243]
	GetHandleVerifier [0x00007FF63772FC66+815670]
	(No symbol) [0x00007FF6375ED6EF]
	(No symbol) [0x00007FF6375E92B4]
	(No symbol) [0x00007FF6375E9450]
	(No symbol) [0x00007FF6375D81FF]
	BaseThreadInitThunk [0x00007FFBC57C257D+29]
	RtlUserThreadStart [0x00007FFBC702AF28+40]

Error occured while scraping bus details for https://www.redbus.in/bus-ticke

NameError: name 'driver' is not defined

In [3]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,R S YADAV SMART BUS PRIVATE LIMITED,VE A/C Sleeper (2+1),22:15,06h 00m,04:15,4.4,759,24 Seats available
1,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,New Khaira Transport,A/C Seater / Sleeper (2+2),21:00,06h 30m,03:30,1.8,INR 400,23 Seats available
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,Rahul Travels,AshokLeyland Stile A/C,21:00,05h 00m,02:00,1.9,629,21 Seats available
3,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,FlixBus,AC Seater (2+2),23:35,05h 25m,05:00,4.5,INR 258.72,16 Seats available
4,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...,SHRI RAM TRAVELS,A/C Seater (2+2),23:59,05h 16m,05:15,3.9,INR 740,18 Seats available
...,...,...,...,...,...,...,...,...,...,...
298,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - MNP0124,Ordinary Non AC Seater 2+3,22:32,04h 40m,03:12,3.3,INR 371,52 Seats available
299,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - UNO0027,Ordinary Non AC Seater 2+3,22:50,06h 10m,05:00,3.3,INR 371,52 Seats available
300,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - MNP0128,Ordinary Non AC Seater 2+3,22:57,04h 40m,03:37,3.3,INR 371,52 Seats available
301,Etah (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/etah-up-to-d...,UPSRTC - MHB0265,Ordinary Non AC Seater 2+3,23:09,05h 51m,05:00,3.3,INR 364,52 Seats available
